In [1]:
%load_ext autoreload
%autoreload 2

In [136]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import requests
import json
import datetime
from collections import OrderedDict

from models.seir.seir_testing import SEIR_Testing
from utils.data import get_global_data, get_indian_data
from utils.plotting import create_plots
from utils.generic import init_params

In [129]:
df_tested, df_statewise, df_state_time_series, df_india_time_series, df_districtwise, df_raw_data, df_travel_history = get_indian_data()

In [135]:
# Imperfecting Testing (How do the curves vary with decreasing sensitivity)

states_time_matrices = []

vanilla_params, testing_params, state_init_values = init_params(N=1e9, init_infected=1)
solver = SEIR_Testing(vanilla_params, testing_params, state_init_values)
sol = solver.solve_ode(method='RK23')
states_time_matrices.append( (sol.y*vanilla_params['N']).astype('int') )

vanilla_params, testing_params, state_init_values = init_params(N=1e9, init_infected=1)
solver = SEIR_Testing(vanilla_params, testing_params, state_init_values)
sol = solver.solve_ode(method='Radau')
states_time_matrices.append( (sol.y*vanilla_params['N']).astype('int') )

vanilla_params, testing_params, state_init_values = init_params(N=1e9, init_infected=1)
solver = SEIR_Testing(vanilla_params, testing_params, state_init_values)
sol = solver.solve_ode(method='LSODA')
states_time_matrices.append( (sol.y*vanilla_params['N']).astype('int') )

vanilla_params, testing_params, state_init_values = init_params(N=1e9, init_infected=1)
solver = SEIR_Testing(vanilla_params, testing_params, state_init_values)
sol = solver.solve_ode(method='BDF')
states_time_matrices.append( (sol.y*vanilla_params['N']).astype('int') )

vanilla_params, testing_params, state_init_values = init_params(R0=2.42, T_inc=1.38, T_inf=3, N=1.12374333e8, init_infected=2)
solver = SEIR_Testing(vanilla_params, testing_params, state_init_values)
sol = solver.solve_ode(time_step=1, method='Radau')
states_time_matrices.append( (sol.y*vanilla_params['N']).astype('int') )

# last_time = 28
create_plots(sol, np.array(states_time_matrices), labels = ['Radau'], savefig=False, filenames=None)

# ind = sol.t[:last_time]
# states_time_matrix = np.array(states_time_matrices)[0, :, :last_time]
# fig, ax = plt.subplots(figsize=(12, 12))
# ax.plot(np.array(df_state_time_series[df_state_time_series['state'] == 'Maharashtra']['date']), 
#         states_time_matrix[2] + states_time_matrix[4] + states_time_matrix[5] + 
#         states_time_matrix[6] + states_time_matrix[7] + states_time_matrix[8] + 
#         states_time_matrix[9] + states_time_matrix[10], label='Predicted')
# ax.plot(np.array(df_state_time_series[df_state_time_series['state'] == 'Maharashtra']['date']), 
#         df_state_time_series[df_state_time_series['state'] == 'Maharashtra']['confirmedCases'], label='True')
# ax.yaxis.set_minor_locator(plt.MultipleLocator(25))
# ax.xaxis.set_major_locator(mdates.DayLocator(interval=7))
# ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))
# ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
# plt.gcf().autofmt_xdate()
# plt.ylabel('No of people')
# plt.xlabel('Date')
# plt.legend()
# plt.grid()
# plt.title("Total No of infections (Active + Recovered + Dead) (Maharashtra) : T_inc = 1.38 days")
# plt.show()

ValueError: could not broadcast input array from shape (11,401) into shape (11)

In [ ]:
states_time_matrices = []
vanilla_params, testing_params, state_init_values = init_params(R0=2.38, T_inf=1, N=1.6787941e7, init_infected=2)
solver = SEIR_Testing(vanilla_params, testing_params, state_init_values)
sol = solver.solve_ode(time_step=1, method='Radau')
states_time_matrices.append( (sol.y*vanilla_params['N']).astype('int') )

last_time = 27
# create_plots(sol, np.array(states_time_matrices)[:, :, :last_time], labels = ['Radau'], last_time=last_time, savefig=False, filenames=None)

ind = sol.t[:last_time]
states_time_matrix = np.array(states_time_matrices)[0, :, :last_time]
fig, ax = plt.subplots(figsize=(12, 12))
ax.plot(df[df['loc'] == 'Delhi']['date'], states_time_matrix[2] + states_time_matrix[4] + states_time_matrix[5] + 
         states_time_matrix[6] + states_time_matrix[7] + states_time_matrix[8] + 
         states_time_matrix[9] + states_time_matrix[10], label='Predicted')
ax.plot(df[df['loc'] == 'Delhi']['date'], df[df['loc'] == 'Delhi']['confirmedCases'], label='True')
ax.xaxis.set_major_locator(mdates.DayLocator(interval=7))
# ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.ylabel('No of people')
plt.xlabel('Days')
plt.legend()
plt.title("Total No of infections (Active + Recovered + Dead) (Delhi)")
plt.show()
